In [80]:
# load in data
reviews=[]
sentiment_ratings=[]
product_types=[]
helpfulness_ratings=[]

with open("Compiled_Reviews.txt") as f:
   for line in f.readlines()[1:]:
        fields = line.rstrip().split('\t')
        reviews.append(fields[0])
        sentiment_ratings.append(fields[1])
        product_types.append(fields[2])
        helpfulness_ratings.append(fields[3])


In [81]:
list(set(sentiment_ratings))

['negative', 'positive']

In [82]:
# LOAD IN STOPWORDS
f = open("data/stopwords.txt", "r")
stopwords = []
for line in f:
    stopwords.append(line.strip())
f.close()

punct = ['#', '"', '""', '%', '$', '&', ')', '(', '+', '*', '-'] 



In [83]:
import re
from collections import Counter
# VOCAB IS 5000 MOST COMMON WORDS

# tokenise reviews
token_def = re.compile("[^ \.?!:,)(\"]+")
tokenized_sents = [token_def.findall(txt) for txt in reviews]
tokens=[]
for s in tokenized_sents:
      tokens.extend([t.lower() for t in s]) # lowercase to avoid uppercase-lowercase reptitions

tokens = [t for t in tokens if t not in stopwords and t not in punct]

# use a counter to count the tokens
counts=Counter(tokens)
# sort the tokens
so=sorted(counts.items(), key=lambda item: item[1], reverse=True)
so=list(zip(*so))[0]

# get 5000 most common into vocabulary
type_list=so[0:5000]

vocab_list = type_list

In [84]:
import numpy as np
# # UNCOMMENT TO RECREATE M
# # CREATE A ONE HOT ENCODED MATRIX FOR EACH REVIEW
# M = np.zeros((len(reviews), len(vocab_list)))
# print(len(reviews))
# for i, rev in enumerate(reviews):
#     if i%10 == 0:
#          print(i)
#     tokens = token_def.findall(rev)
    
#     # iterate over vocab
#     for j, vocab_token in enumerate(vocab_list):
#         # if the current word j occurs in the current review i then set the matrix element at i,j to be one. Otherwise leave as zero.
#         if vocab_token in tokens:
#               M[i,j] += 1

M = np.load('../copy/M.npy')

In [85]:
# GET TRAINING AND TESTING DATA
# train_ints=np.random.choice(len(reviews),int(len(reviews)*0.8),replace=False)
# test_ints=list(set(range(0,len(reviews))) - set(train_ints))
# M_train = M[train_ints,]
# M_test = M[test_ints,]

# # for labels, use a vector representation
# labels_train = [sentiment_ratings[i] for i in train_ints]
# labels_test = [sentiment_ratings[i] for i in test_ints]

# including dev set
train_ints=np.random.choice(len(reviews),int(len(reviews)*0.8),replace=False)
remaining_ints=list(set(range(0,len(reviews))) - set(train_ints))
test_ints=np.random.choice(len(remaining_ints),int(len(remaining_ints)*0.5),replace=False)
dev_ints=list(set(range(0,len(remaining_ints))) - set(test_ints))

M_train = M[train_ints,]
M_test = M[test_ints,]
M_dev = np.array(M[dev_ints,])

# for labels, use a vector representation
labels_train = [sentiment_ratings[i] for i in train_ints]
labels_test = [sentiment_ratings[i] for i in test_ints]
labels_dev = [sentiment_ratings[i] for i in dev_ints]

In [125]:
import random
indices = list(range(len(reviews)))
random.seed(42)  # For reproducibility
random.shuffle(indices)

print(indices)

# Define the split ratios
train_size = 0.8  # 80% for training
dev_size = 0.1    # 10% for development
test_size = 0.1   # 10% for testing

# Calculate the split indices
train_end = int(len(indices) * train_size)
dev_end = train_end + int(len(indices) * dev_size)

# Create the index splits
train_indexes = indices[:train_end]
dev_indexes = indices[train_end:dev_end]
test_indexes = indices[dev_end:]

M_train = M[train_indexes,]
M_test = M[test_indexes,]
M_dev = np.array(M[dev_indexes,])

# for labels, use a vector representation
labels_train = [sentiment_ratings[i] for i in train_indexes]
labels_test = [sentiment_ratings[i] for i in test_indexes]
labels_dev = [sentiment_ratings[i] for i in dev_indexes]

[17863, 29826, 13349, 21718, 14817, 35760, 18612, 35888, 4742, 9883, 28532, 20993, 25472, 10160, 16855, 31839, 2577, 36308, 9067, 18692, 22039, 13515, 25659, 14756, 13712, 18409, 22041, 5750, 28781, 11159, 15819, 28658, 1504, 35437, 6504, 29509, 20690, 12013, 23655, 26266, 9781, 29079, 29351, 13628, 4767, 27239, 21247, 27471, 14918, 25345, 16130, 27436, 16620, 36112, 20081, 7381, 6173, 35247, 2589, 16545, 30382, 3657, 17023, 12966, 2221, 29789, 34618, 3586, 14246, 20057, 8945, 28232, 15480, 34146, 35831, 33528, 34003, 4016, 12883, 27395, 20382, 22788, 22438, 35571, 24383, 19697, 25869, 7986, 19398, 25410, 31110, 25651, 27499, 28565, 9933, 24085, 3671, 3499, 35194, 29046, 34707, 10735, 16744, 12564, 31903, 29487, 7485, 29563, 7594, 32713, 21192, 30846, 29967, 32561, 4372, 21396, 69, 6137, 21939, 4600, 25787, 25765, 14902, 12537, 867, 18964, 36506, 25473, 11762, 23386, 31120, 29244, 6532, 1266, 19288, 2238, 36382, 17268, 23349, 61, 5503, 31526, 12571, 25696, 33518, 10200, 2742, 21733, 30

In [126]:
# y has to be an vector of integers (0 for negative 1 for positive)
y=np.array([int(l == "positive") for l in labels_train])

In [72]:
# TRYING STUFF OUT
num_features = 2
hidden_size = 100
w_0_1 = np.random.randn(num_features, hidden_size)
w_1_2 = np.random.randn(hidden_size, 1)
print(w_0_1)
print('')
print(w_1_2)

inputs = [0, 0, 1]
# multiply the diff by the relu outputs of the hidden nodes
# take away the current weight values by this number * lr
hidden_outputs = np.array([0, 0, 0.28])
diff = 0.72

hidden_outputs.dot(diff)

# multiply diff by the output weights in the hidden layer to get the errors of the hidden nodes
# multiply the hidden node errors by the derivatives of the hidden layer outputs
# multiply this by the inputs
# take the current weights away from this * the learning rate
# so you do
# diff * hidden weight * relu deriv of hidden layer input * input to that weight

relu_deriv = [1 if x > 0 else 0 for x in hidden_outputs]
hidden_outputs.dot(diff) * relu_deriv * inputs

[[-1.02796597e+00  5.04212798e-01  1.50817625e+00  9.98639854e-02
   5.60563290e-01  3.01001809e-01 -2.01753786e-01 -8.98157612e-01
   1.43334761e+00 -9.59305502e-01  1.33693722e+00 -3.16833571e-01
  -9.90014419e-01 -6.52381250e-01 -9.17021789e-01 -9.34164143e-01
  -7.58829974e-01 -1.45778291e+00 -1.88204342e+00  7.26514275e-01
   4.17528476e-01  3.25178146e-01 -1.91949215e+00  1.45230799e-02
   1.26140277e+00  8.06619050e-01 -1.46470263e+00 -1.70277512e-01
   1.38113546e+00  9.07627009e-01  1.19979579e+00 -7.12260079e-01
  -1.00286310e+00 -7.67504274e-01 -2.55600219e-01 -2.02015605e+00
  -1.41777380e+00 -9.62367014e-02 -3.40449674e-01  7.75107467e-01
   8.26606638e-01 -8.93710975e-01  1.69528662e-01  1.00339550e+00
  -8.28921214e-01  1.50067041e+00  5.21580610e-01  5.00569539e-01
   3.24802537e-01 -4.51022166e-01 -6.40728577e-01  1.15103768e+00
   1.58351731e+00  5.28788499e-01  7.87233259e-01  1.10565640e+00
  -9.51775567e-02  1.64469742e-01  3.22095091e-01  1.18050160e+00
   1.26290

array([0.    , 0.    , 0.2016])

In [134]:
import matplotlib.pyplot as plt
# initialise variables
num_features=len(vocab_list)
hidden_size = 3000
weights_0_1 = np.random.randn(num_features, hidden_size)
weights_1_2 = np.random.randn(hidden_size, 1)
bias_1 = np.random.randn(hidden_size)  # Shape: (100,)
bias_2 = np.random.rand(1)  # Shape: (1,)
n_iters = 2500
lr=0.01
logistic_loss=[]
num_samples=len(y)

for i in range(n_iters):
    print(i)

    # forward pass
    layer_1 = np.maximum(M_train.dot(weights_0_1) + bias_1, 0)
    layer_2 = layer_1.dot(weights_1_2) + bias_2
    q = 1/(1+np.exp(-layer_2))

    # difference between actual labels
    diff = (q.T - y)


    loss = -np.mean(y * np.log(q.T) + (1 - y) * np.log(1 - q.T))
    logistic_loss.append(loss)

    # for hidden layer weights
    grad_weights_1_2 = layer_1.T.dot(diff.T) / num_samples
    grad_bias_2 = np.sum(diff) / num_samples

    # for input layer weights
    layer_1_error = diff.T.dot(weights_1_2.T) # errors at hidden nodes
    layer_1_delta = layer_1_error * (layer_1 > 0) # multiply by relu derivative
    grad_weights_0_1 = M_train.T.dot(layer_1_delta) / num_samples # multiply by inputs
    grad_bias_1 = np.sum(layer_1_delta, axis=0) / num_samples

    # update weights
    weights_0_1 -= lr * grad_weights_0_1
    weights_1_2 -= lr * grad_weights_1_2
    bias_1 -= lr * grad_bias_1
    bias_2 -= lr * grad_bias_2

    # evaluate on dev set
    if i%10 == 0:
        print(f'EVALUATING ON DEV SET: ITERATION {i}')

        layer_1 = np.maximum(M_dev.dot(weights_0_1) + bias_1, 0) 
        layer_2 = layer_1.dot(weights_1_2) + bias_2
        q = 1/(1+np.exp(-layer_2))

        y_dev_pred = [int(prob > 0.5) for prob in q] # if sigmoid percentage > 0.5, output 1, else output 0
        y_dev=[int(l == "positive") for l in labels_dev]

        acc_test=[int(yp == y_dev[s]) for s,yp in enumerate(y_dev_pred)]
        print(f'accuracy: {sum(acc_test)/len(acc_test)}')

        # precision and recall
        labels_dev_pred=["positive" if s == 1 else "negative" for s in y_dev_pred]
        true_positives=sum([int(yp == "positive" and labels_test[s] == "positive") for s,yp in enumerate(labels_dev_pred)])
        false_negatives=sum([int(yp == "negative" and labels_test[s] == "positive") for s,yp in enumerate(labels_dev_pred)])
        false_positives=sum([int(yp == "positive" and labels_test[s] == "negative") for s,yp in enumerate(labels_dev_pred)])
        true_negatives=sum([int(yp == "negative" and labels_test[s] == "negative") for s,yp in enumerate(labels_dev_pred)])

        precision = true_positives/(true_positives + false_positives)
        recall = true_positives/(true_positives + false_negatives)
        print(f'precision: {precision}')
        print(f'recall: {recall}')
        print('')

plt.plot(range(1,n_iters),logistic_loss[1:])
plt.xlabel("number of epochs")
plt.ylabel("loss")
#loss = sum(-(np.ones(len(y))*np.log2(q)+(np.ones(len(y))-y)*np.log2(np.ones(len(y))-q)))

0


/var/folders/y9/j91d15zx5f31r234bqr2767h0000gn/T/ipykernel_15232/2805317974.py:20: RuntimeWarning: overflow encountered in exp
  q = 1/(1+np.exp(-layer_2))
/var/folders/y9/j91d15zx5f31r234bqr2767h0000gn/T/ipykernel_15232/2805317974.py:26: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(y * np.log(q.T) + (1 - y) * np.log(1 - q.T))
/var/folders/y9/j91d15zx5f31r234bqr2767h0000gn/T/ipykernel_15232/2805317974.py:26: RuntimeWarning: invalid value encountered in multiply
  loss = -np.mean(y * np.log(q.T) + (1 - y) * np.log(1 - q.T))


EVALUATING ON DEV SET: ITERATION 0


/var/folders/y9/j91d15zx5f31r234bqr2767h0000gn/T/ipykernel_15232/2805317974.py:51: RuntimeWarning: overflow encountered in exp
  q = 1/(1+np.exp(-layer_2))
/var/folders/y9/j91d15zx5f31r234bqr2767h0000gn/T/ipykernel_15232/2805317974.py:53: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_dev_pred = [int(prob > 0.5) for prob in q] # if sigmoid percentage > 0.5, output 1, else output 0


accuracy: 0.48357963875205257
precision: 0.5787545787545788
recall: 0.3783524904214559

1
2
3
4
5
6
7
8
9
10
EVALUATING ON DEV SET: ITERATION 10
accuracy: 0.5112205801860974
precision: 0.5771195097037793
recall: 0.5411877394636015

11
12
13
14
15
16
17
18
19
20
EVALUATING ON DEV SET: ITERATION 20
accuracy: 0.5197044334975369
precision: 0.575772934617334
recall: 0.5440613026819924

21
22
23
24
25
26
27
28
29
30
EVALUATING ON DEV SET: ITERATION 30
accuracy: 0.5287356321839081
precision: 0.5743927125506073
recall: 0.5435823754789272

31
32
33
34
35
36
37
38
39
40
EVALUATING ON DEV SET: ITERATION 40
accuracy: 0.5394088669950738
precision: 0.5737868934467234
recall: 0.5493295019157088

41
42
43
44
45
46
47
48
49
50
EVALUATING ON DEV SET: ITERATION 50
accuracy: 0.5498084291187739
precision: 0.573697270471464
recall: 0.553639846743295

51
52
53
54
55
56
57
58
59
60
EVALUATING ON DEV SET: ITERATION 60
accuracy: 0.5599343185550082
precision: 0.5749506903353058
recall: 0.5584291187739464

61
62


KeyboardInterrupt: 

In [136]:

for i in range(n_iters):
    print(i)

    # forward pass
    layer_1 = np.maximum(M_train.dot(weights_0_1) + bias_1, 0)
    layer_2 = layer_1.dot(weights_1_2) + bias_2
    q = 1/(1+np.exp(-layer_2))

    # difference between actual labels
    diff = (q.T - y)


    loss = -np.mean(y * np.log(q.T) + (1 - y) * np.log(1 - q.T))
    logistic_loss.append(loss)

    # for hidden layer weights
    grad_weights_1_2 = layer_1.T.dot(diff.T) / num_samples
    grad_bias_2 = np.sum(diff) / num_samples

    # for input layer weights
    layer_1_error = diff.T.dot(weights_1_2.T) # errors at hidden nodes
    layer_1_delta = layer_1_error * (layer_1 > 0) # multiply by relu derivative
    grad_weights_0_1 = M_train.T.dot(layer_1_delta) / num_samples # multiply by inputs
    grad_bias_1 = np.sum(layer_1_delta, axis=0) / num_samples

    # update weights
    weights_0_1 -= lr * grad_weights_0_1
    weights_1_2 -= lr * grad_weights_1_2
    bias_1 -= lr * grad_bias_1
    bias_2 -= lr * grad_bias_2

    # evaluate on dev set
    if i%10 == 0:
        print(f'EVALUATING ON DEV SET: ITERATION {i}')

        layer_1 = np.maximum(M_dev.dot(weights_0_1) + bias_1, 0) 
        layer_2 = layer_1.dot(weights_1_2) + bias_2
        q = 1/(1+np.exp(-layer_2))

        y_dev_pred = [int(prob > 0.5) for prob in q] # if sigmoid percentage > 0.5, output 1, else output 0
        y_dev=[int(l == "positive") for l in labels_dev]

        acc_test=[int(yp == y_dev[s]) for s,yp in enumerate(y_dev_pred)]
        print(f'accuracy: {sum(acc_test)/len(acc_test)}')

        # precision and recall
        labels_dev_pred=["positive" if s == 1 else "negative" for s in y_dev_pred]
        true_positives=sum([int(yp == "positive" and labels_test[s] == "positive") for s,yp in enumerate(labels_dev_pred)])
        false_negatives=sum([int(yp == "negative" and labels_test[s] == "positive") for s,yp in enumerate(labels_dev_pred)])
        false_positives=sum([int(yp == "positive" and labels_test[s] == "negative") for s,yp in enumerate(labels_dev_pred)])
        true_negatives=sum([int(yp == "negative" and labels_test[s] == "negative") for s,yp in enumerate(labels_dev_pred)])

        precision = true_positives/(true_positives + false_positives)
        recall = true_positives/(true_positives + false_negatives)
        print(f'precision: {precision}')
        print(f'recall: {recall}')
        print('')

plt.plot(range(1,n_iters),logistic_loss[1:])
plt.xlabel("number of epochs")
plt.ylabel("loss")

0


/var/folders/y9/j91d15zx5f31r234bqr2767h0000gn/T/ipykernel_15232/577603185.py:7: RuntimeWarning: overflow encountered in exp
  q = 1/(1+np.exp(-layer_2))
/var/folders/y9/j91d15zx5f31r234bqr2767h0000gn/T/ipykernel_15232/577603185.py:13: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(y * np.log(q.T) + (1 - y) * np.log(1 - q.T))
/var/folders/y9/j91d15zx5f31r234bqr2767h0000gn/T/ipykernel_15232/577603185.py:13: RuntimeWarning: invalid value encountered in multiply
  loss = -np.mean(y * np.log(q.T) + (1 - y) * np.log(1 - q.T))


EVALUATING ON DEV SET: ITERATION 0


/var/folders/y9/j91d15zx5f31r234bqr2767h0000gn/T/ipykernel_15232/577603185.py:38: RuntimeWarning: overflow encountered in exp
  q = 1/(1+np.exp(-layer_2))
/var/folders/y9/j91d15zx5f31r234bqr2767h0000gn/T/ipykernel_15232/577603185.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_dev_pred = [int(prob > 0.5) for prob in q] # if sigmoid percentage > 0.5, output 1, else output 0


accuracy: 0.5821018062397373
precision: 0.5739385065885798
recall: 0.5632183908045977

1
2
3
4
5
6
7
8
9
10
EVALUATING ON DEV SET: ITERATION 10
accuracy: 0.5889436234263821
precision: 0.5730282375851996
recall: 0.5636973180076629

11
12
13
14
15
16
17
18
19
20
EVALUATING ON DEV SET: ITERATION 20
accuracy: 0.5925013683634374
precision: 0.5699183085055262
recall: 0.5680076628352491

21
22
23
24
25
26
27
28
29
30
EVALUATING ON DEV SET: ITERATION 30
accuracy: 0.5974274767378216
precision: 0.5698511761881901
recall: 0.5684865900383141

31
32
33
34
35
36
37
38
39
40
EVALUATING ON DEV SET: ITERATION 40
accuracy: 0.5998905309250137
precision: 0.5714285714285714
recall: 0.5708812260536399

41
42
43
44
45
46
47
48
49
50
EVALUATING ON DEV SET: ITERATION 50
accuracy: 0.6075533661740559
precision: 0.5720419847328244
recall: 0.5742337164750958

51
52
53
54
55
56
57
58
59
60
EVALUATING ON DEV SET: ITERATION 60
accuracy: 0.6105637657361795
precision: 0.5727229375298045
recall: 0.5751915708812261

61
6

KeyboardInterrupt: 

In [78]:
# evaluate on test set
layer_1 = np.maximum(M_test.dot(weights_0_1) + bias_1, 0) 
layer_2 = layer_1.dot(weights_1_2) + bias_2
q = 1/(1+np.exp(-layer_2))

y_test_pred = [int(prob > 0.5) for prob in q] # if sigmoid percentage > 0.5, output 1, else output 0
print(y_test_pred)
y_test=[int(l == "positive") for l in labels_test]
print(y_test)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

/var/folders/y9/j91d15zx5f31r234bqr2767h0000gn/T/ipykernel_15232/2603051726.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_test_pred = [int(prob > 0.5) for prob in q] # if sigmoid percentage > 0.5, output 1, else output 0


In [79]:
print(sum(y_test_pred)/len(y_test_pred))
print(sum(y_test)/len(y_test))

1.0
1.0


In [59]:
# EVALUATION
# accuracy
acc_test=[int(yp == y_test[s]) for s,yp in enumerate(y_test_pred)]
print(f'accuracy: {sum(acc_test)/len(acc_test)}')

# precision and recall
labels_test_pred=["positive" if s == 1 else "negative" for s in y_test_pred]
true_positives=sum([int(yp == "positive" and labels_test[s] == "positive") for s,yp in enumerate(labels_test_pred)])
false_negatives=sum([int(yp == "negative" and labels_test[s] == "positive") for s,yp in enumerate(labels_test_pred)])
false_positives=sum([int(yp == "positive" and labels_test[s] == "negative") for s,yp in enumerate(labels_test_pred)])
true_negatives=sum([int(yp == "negative" and labels_test[s] == "negative") for s,yp in enumerate(labels_test_pred)])

precision = true_positives/(true_positives + false_positives)
recall = true_positives/(true_positives + false_negatives)
print(f'precision: {precision}')
print(f'recall: {recall}')


accuracy: 0.5835841313269494
precision: 0.9241460541813898
recall: 0.5903686982693754


In [60]:
# EXAMINING WEIGHTS
print("most impactful words for a positive review:")
print([vocab_list[x] for x in np.argsort(weights_0_1[:,1])[::-1][0:20]])

print("\nmost impactful words for a negative review:")
print([vocab_list[x] for x in np.argsort(weights_0_1[:,1])[0:20]])


most impactful words for a positive review:
['move', 'scent', 'sturdy', 'weeks', 'fabric', 'poor', 'puzzle', '360', 'love', 'rack', 'subscribed', 'window', 'regarding', 'episode', 'feel', "i've", 'dollars', 'leg', 'treat', 'prefer']

most impactful words for a negative review:
['im', 'gave', 'knife', 'stiff', 'force', 'soon', 'list', 'comedy', 'hoping', 'pens', 'signal', 'norton', 'naturally', 'pleased', 'pot', 'forever', 'part', 'century', 'mac', 'acting']
